In [3]:
from datasets import load_dataset
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_sequence, PackedSequence, unpack_sequence, pad_sequence
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
import torch.optim as optim
from transformers import AutoTokenizer


# ds = load_dataset("abisee/cnn_dailymail", "3.0.0")
ds = load_dataset("yhavinga/ccmatrix", "en-de", streaming=True)
ds

IterableDatasetDict({
    train: IterableDataset({
        features: ['id', 'score', 'translation'],
        n_shards: 1
    })
})

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [7]:
next(iter(ds['train']))

{'id': 0,
 'score': 1.2499472,
 'translation': {'en': 'Or do they think that We hear not their secrets and their private counsels?',
  'de': 'Oder meinen sie, daß Wir ihre Geheimnisse und heimlichen Beratungen nicht hören?'}}

In [8]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'bos_token': '<|beginingoftext|>'})

1

In [9]:
tokenizer.special_tokens_map

{'bos_token': '<|beginingoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [7]:
hl = [(d['highlights']) for d in ds['validation']]

In [8]:
len(tokenizer(list(sorted(hl, key=len, reverse=True))[1])['input_ids'])

751

In [9]:
tokenizer(ds['train'][0]['article'])

{'input_ids': [43, 47383, 11, 4492, 357, 12637, 8, 1377, 5850, 14179, 3491, 7806, 5325, 33783, 8810, 1895, 284, 257, 2098, 4248, 1238, 1510, 7198, 3901, 13, 16, 1510, 8, 15807, 355, 339, 4962, 1248, 319, 3321, 11, 475, 339, 17424, 262, 1637, 1839, 470, 3350, 257, 4822, 319, 683, 13, 7806, 5325, 33783, 355, 5850, 14179, 287, 366, 18308, 14179, 290, 262, 8284, 286, 262, 9643, 1, 1675, 262, 18641, 286, 30914, 5721, 1023, 1088, 262, 995, 11, 262, 1862, 8674, 1139, 339, 468, 645, 3352, 284, 277, 799, 353, 465, 5003, 1497, 319, 3049, 5006, 11, 4144, 290, 16527, 4671, 13, 366, 40, 836, 470, 1410, 284, 307, 530, 286, 883, 661, 508, 11, 355, 2582, 355, 484, 1210, 1248, 11, 6451, 2822, 2405, 257, 4858, 5701, 1097, 4947, 393, 1223, 2092, 553, 339, 1297, 281, 6638, 39877, 2961, 428, 1227, 13, 366, 40, 836, 470, 892, 314, 1183, 307, 3573, 45997, 13, 366, 464, 1243, 314, 588, 7067, 389, 1243, 326, 1575, 546, 838, 8059, 1377, 3835, 290, 36731, 290, 35962, 526, 1629, 1248, 11, 5325, 33783, 481, 307, 1

## Model

In [10]:
tokenizer.special_tokens_map

{'bos_token': '<|beginingoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SOS_token = tokenizer.special_tokens_map['bos_token']
SOS_token = tokenizer.vocab[SOS_token]
EOS_token = tokenizer.special_tokens_map['eos_token']
EOS_token = tokenizer.vocab[EOS_token]
MAX_LENGTH = 800

def simple_elementwise_apply(fn, packed_sequence):
    """applies a pointwise function fn to each element in packed_sequence"""
    return torch.nn.utils.rnn.PackedSequence(fn(packed_sequence.data), packed_sequence.batch_sizes)

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        if isinstance(input, PackedSequence):
          embedded = simple_elementwise_apply(lambda x: self.dropout(self.embedding(x)), input)
        else:
          embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden

        if target is not None:
          decoder_outputs = self.forward_teacher(decoder_input, decoder_hidden, target)
          # decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
          decoder_outputs = simple_elementwise_apply(lambda x: F.log_softmax(x, dim=-1), decoder_outputs)
          return decoder_outputs, None, None

        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            # Without teacher forcing: use its own predictions as the next input
            _, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_teacher(self, decoder_input, decoder_hidden, target_packed):
      i = 0
      decoder_outputs = []
      for batch_size in target_packed.batch_sizes:
        # print(batch_size)
        decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
        decoder_outputs.append(decoder_output[:batch_size])
        # decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
        decoder_input = target_packed.data[i:i + batch_size].unsqueeze(-1)
        decoder_hidden = decoder_hidden[:, :batch_size, :]
        # end
        i += batch_size
      decoder_outputs = torch.nn.utils.rnn.PackedSequence(torch.cat(decoder_outputs), target_packed.batch_sizes)
      return decoder_outputs


    def forward_step(self, input, hidden):
        # print(input.shape, hidden.shape)
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

## Training loop

In [12]:
def decode(log_probs):
  indices = log_probs.topk(k=1, dim=-1).indices.squeeze(-1)
  if len(indices.shape) == 1:
    return tokenizer.decode(indices)
  return [tokenizer.decode(s) for s in indices]


In [13]:
hidden_size = 1024
lr = 1e-4

enc = EncoderRNN(len(tokenizer), hidden_size).to(device)
dec = DecoderRNN(hidden_size, len(tokenizer)).to(device)
enc_optimizer = optim.Adam(enc.parameters(), lr)
dec_optimizer = optim.Adam(dec.parameters(), lr)

In [20]:
loss_f = nn.NLLLoss()
bs = 8
train_dataloader = DataLoader(ds['train'], batch_size=bs, shuffle=True)

epochs = 1
for e in range(epochs):
  for i, data in tqdm(enumerate(train_dataloader)):
    enc_optimizer.zero_grad()
    dec_optimizer.zero_grad()
    art, summ = data['article'], data['highlights']
    art = tokenizer(art)['input_ids']
    summ = tokenizer(summ)['input_ids']
    for s in summ:
      s.append(EOS_token)
    art_lens_desc_idx = np.argsort([len(a) for a in art])[::-1]
    reverse_art_sort = np.argsort(art_lens_desc_idx) 
    summ_lens_desc_idx = np.argsort([len(a) for a in summ])[::-1]
    art_to_summ_indexes = reverse_art_sort[summ_lens_desc_idx]
    art_packed = pack_sequence([torch.tensor(art[i]) for i in art_lens_desc_idx]).to(device)
    target_packed = pack_sequence([torch.tensor(summ[i]) for i in summ_lens_desc_idx]).to(device)

    enc_out, enc_hid = enc(art_packed)
    enc_out = unpack_sequence(enc_out)
    enc_out = torch.stack([o[-1] for o in enc_out])
    enc_out = enc_out[art_to_summ_indexes]
    enc_hid = enc_hid[:, art_to_summ_indexes, :]
      
    dec_out, _, _ = dec(enc_out, enc_hid, target_packed)
      
    loss = loss_f(
            dec_out.data.view(-1, dec_out.data.size(-1)),
            target_packed.data.view(-1)
    )
    loss.backward()
    enc_optimizer.step()
    dec_optimizer.step()
      
    with torch.no_grad():
      if i % 100 == 0:
        print(f"loss={loss}")
        print("\nArticle:")
        print(tokenizer.decode(unpack_sequence(art_packed)[art_to_summ_indexes[0]])[:1000])
        print("\nTarget:")
        print(tokenizer.decode(unpack_sequence(target_packed)[0]))
        print("\nPredicted:")
        print(decode(unpack_sequence(dec_out)[0][:,0,:]))


1it [00:01,  1.33s/it]

loss=5.454220294952393

Article:
By. Daily Mail Reporter. Serial philanderer Robert F. Kennedy Jr. is reported to have kept a list of dozens of women filed under the letter ‘G’ in his cell phone, including his soon-to-be-wife actress Cheryl Hines and Chelsea Kirwan, his alleged mistress who is currently involved in a bitter divorce battle. Allegations about a cell phone full of women’s numbers, have emerged at a tricky time for the 60-year-old, twice-married, father-of-six. Later this month he is set to marry Curb Your Enthusiasm star Hines, 48, in a ceremony expected to take place over the weekend of July 25, during the annual Kennedy family reunion in Hyannis Port, Massachusetts. Robert Kennedy Jr, 60, is set to marry Curb Your Enthusiasm star Hines, 48, in a ceremony expected to take place over the weekend of July 25, during the annual Kennedy family reunion in Hyannis Port, Massachusetts. RFK is also in the potentially embarrassing situation of being deposed on July 22 by lawyers f

101it [01:28,  1.10it/s]

loss=5.671022891998291

Article:
Notorious'serial stowaway' Marilyn Jean Hartman has struck again. The 63-year-old California resident was arrested Monday in Jacksonville, Florida, after she allegedly snuck onto a flight from Minneapolis Sunday by using someone else's identity. Once in Jacksonville, the woman allegedly used a fake name, Maria Sandgren, to get transportation to Omni Amelia Island Plantation Resort, where she checked into a $300-a-night villa. But her triumph was short-lived: Hartman fled hours later when the real guest showed up at the hotel. She's baaack! Notorious'serial stowaway' Marilyn Jean Hartman, 63, was arrested in Florida after allegedly sneaking onto a flight and then checking into a posh resort using someone else's name. The fleet-footed senior citizen was discovered by security guards the following day hiding in a vacant room that was been renovated, reported NBC News. Hartman has been charged with felony fraud, defrauding an innkeeper and misdemeanor tresp

201it [02:51,  1.26it/s]

...

loss=5.1926374435424805

Article:
The PM knows that Nando’s — the fast food chain that’s spreading across the country like wildfire — crosses class, political and ethnic divisions. Forget the NHS, tax cuts or TV debates. The way to win British people’s hearts at the next election is through their stomachs. When David Cameron told a hip youth radio station this week he’d like to take world leaders to lunch at Nando’s — rather than a Harvester or a Gordon Ramsay restaurant — he knew what he was talking about. Not only has he visited a Bristol outlet of Nando’s — the fast food chicken chain that’s spreading across the country like wildfire — but he also knows that the restaurant crosses class, political and ethnic divisions. Whoever gets the Nando’s vote at the election will sweep the nation. There are 333 Nando’s outlets in Britain; 1,000 in all, in 35 countries. Main markets are Britain, Australia and South Africa. But Nando’s is big in America, too. When President Obama spoke at the Un

13901it [3:18:47,  1.21it/s]

loss=5.190968990325928

Article:
By. Lucy Crossley. An NHS worker defrauded a leading cancer hospital out of more than £640,000 and spent the money meant for life-saving drugs on Gucci handbags and her wedding, a court heard. Stacey Tipler, 32, is accused of exploiting her position in the accounts department of the Royal Marsden Hospital Trust in London to alter invoices and divert £642,827.98 that should have been paid to the hospital’s suppliers. Tipler changed the bank details of genuine suppliers for the hospital, one of the world's top cancer units, to a group of people recruited by her 33-year-old fiancé Scott Chaplin, Southwark Crown Court was told. Hospital: Stacey Tipler, 32, is accused of defrauding the Royal Marsden Hospital Trust in London (pictured) out of more than £640,000 and spending the money on handbags and her wedding. It is alleged that once the money had been transferred it was then withdrawn and split among gang members, with Tipler using her share to buy Gucci h

13964it [3:19:42,  1.17it/s]


KeyboardInterrupt: 

In [22]:
torch.save(dec, "dec.w")